In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings

# Suprimindo os warnings temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# CONSTANTS
DELAY = 3
SEASON_RANGE = list(range(2023, 2015, -1))
URL = f"https://fbref.com/en/comps/12/La-Liga-Stats" # URL constante (usada como base) da premier league, alterar para o Brasileirão, se necessário

In [3]:
url = URL # url não constante, pois será alterada ao longo do fluxo

*2 - CAPTURANDO A TABELA DE INFORMAÇÃO GERAL DOS TIMES (Scores and Fixtures)*

In [4]:
teams_data_table = []
matches_table = []

for year in SEASON_RANGE:
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("#meta > div:nth-child(2) > div")[0].find('a').get("href")
    url = f"https://fbref.com{previous_season}"
    
    # Obtém os dados da página de cada time
    for team_url in team_urls:
        response = requests.get(team_url)

        # Verifica se a solicitação foi bem-sucedida
        try:
            # Extrai os dados da segunda tabela "Scores and Fixtures" em forma de dataframe
            team_data = pd.read_html(response.text, match="Scores & Fixtures")[0]
        
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # Pegamos o nome do time através da url
            season = team_url.split("/")[-2][:9] if year != 2023 else '2022-2023' # Pegamos a temporada através da url

            # Criações de features (colunas)
            team_data["Season"] = season
            team_data["Team"] = team_name

            # Adiciona os dados à lista "teams_data_table"
            teams_data_table.append(team_data)
            matches_table.append(team_data.loc[:, ~team_data.columns.isin(['xG', 'xGA', 'Age','Notes','Match Report'])].rename(columns={"GF":"Goals for", "GA":"Goals against"}))

        except Exception as e:
            print(f"""A solicitação para {team_url} falhou.
                  Erro: {e} \n
                  código de status: {response.status_code}""")
            
        time.sleep(DELAY)

A solicitação para https://fbref.com/en/squads/fc536746/2018-2019/Real-Betis-Stats falhou.
                  Erro: No tables found 

                  código de status: 500


***PARA VISUALIZAR AS TABELAS***

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
  
dict_matchs_table = []
for dataframe in matches_table:
    dict_matchs_table.append(dataframe.to_dict(orient="records"))

In [7]:
from database import create_documents

In [8]:
for match in dict_matchs_table:
    documents = create_documents('matches', match)